In [14]:
# Find subs with 2 or more dates

from pathlib import Path
import re
import json

adnidir = '/run/media/anton/AntonData1/ADNI/fMRI/ADNI'
places = ['Axial_MB_rsfMRI__Eyes_Open_', 'Resting_State_fMRI', 'Axial_rsfMRI__Eyes_Open_', 
          'Axial_fcMRI__EYES_OPEN_', 'Axial_fcMRI__Eyes_Open_', 'Axial_rsfMRI__EYES_OPEN_',
          'Axial_rsfMRI__Eyes_Open__-phase_P_to_A', 'AXIAL_RS_fMRI__EYES_OPEN_',
          'Axial_fcMRI_0_angle__EYES_OPEN_', 'Axial_RESTING_fcMRI__EYES_OPEN_',
          'Extended_Resting_State_fMRI', 'Extended_AXIAL_rsfMRI_EYES_OPEN',
          'Axial_rsFMRI_Eyes_Open', 'Axial_rsfMRI__Eyes_Open__10_min__-PJ']

count = 0
longsubs = []

for sub in Path(adnidir).iterdir():
    if not sub.is_dir():
        continue
    subdirs = []
    for place in places:
        placedir = sub / place
        if not placedir.is_dir():
            continue
        for datedir in placedir.iterdir():
            if not datedir.is_dir():
                continue
            date = re.match('(\d+)-(\d+)-(\d+)_.*', datedir.name)
            if not date:
                continue
            year = date.group(1)
            month = date.group(2)
            day = date.group(3)
            # Two parameters we need along with date
            mniscan = None
            tr = None
            # Random additional I directory level
            for singledir in datedir.iterdir():
                if not singledir.is_dir():
                    continue
                for file in singledir.iterdir():
                    ismniscan = re.match('^wr.*.nii$', file.name)
                    if ismniscan:
                        mniscan = file
                        continue
                    # Must look for json file to to get repetition time
                    isjson = re.match('.*.json$', file.name)
                    if isjson:
                        params = json.load(open(str(file), 'r'))
                        tr = params["RepetitionTime"]
                        continue
                break
            if mniscan and json:
                subdirs.append(dict(year=year, month=month, mniscan=str(mniscan), tr=tr))
    if len(subdirs) > 1:
        print(count, sub.name, subdirs)
        longsubs.append(dict(name=sub.name, scans=subdirs))
        count += 1

print(len(longsubs))

0 002_S_0295 [{'year': '2011', 'month': '06', 'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623/wrI238623_Resting_State_fMRI_20110602075636_501.nii', 'tr': 6.02158}, {'year': '2012', 'month': '05', 'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/002_S_0295/Resting_State_fMRI/2012-05-10_15_42_37.0/I303069/wrI303069_Resting_State_fMRI_20120510154237_501.nii', 'tr': 3.001}]
1 002_S_0413 [{'year': '2019', 'month': '08', 'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/002_S_0413/Axial_MB_rsfMRI__Eyes_Open_/2019-08-27_09_39_37.0/I1221056/wrI1221056_Axial_MB_rsfMRI_(Eyes_Open)_20190827093938_11.nii', 'tr': 0.607}, {'year': '2011', 'month': '06', 'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/002_S_0413/Resting_State_fMRI/2011-06-16_12_18_49.0/I240811/wrI240811_Resting_State_fMRI_20110616121849_601.nii', 'tr': 6.02158}, {'year': '2012', 'month': '05', 'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/00

In [16]:
# Dump subs paths and trs to file

import pickle

pickle.dump(longsubs, open('/home/anton/Documents/Tulane/Research/Work/ADNI/long_sub_scans.pkl', 'wb'))

print('Complete')

Complete


In [1]:
# Load sub paths and trs

import pickle

longsubs = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/ADNI/long_sub_scans.pkl', 'rb'))

print(len(longsubs))

699


In [2]:
longsubs[0]

{'name': '002_S_0295',
 'scans': [{'year': '2011',
   'month': '06',
   'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623/wrI238623_Resting_State_fMRI_20110602075636_501.nii',
   'tr': 6.02158},
  {'year': '2012',
   'month': '05',
   'mniscan': '/run/media/anton/AntonData1/ADNI/fMRI/ADNI/002_S_0295/Resting_State_fMRI/2012-05-10_15_42_37.0/I303069/wrI303069_Resting_State_fMRI_20120510154237_501.nii',
   'tr': 3.001}]}

In [3]:
import numpy as np
from nilearn import image

# Load power template coords
powerfile = '/home/anton/Documents/Tulane/Hackathon/power264/power264MNI.nii.gz'

powerimg = image.load_img(powerfile)
datpower = image.get_data(powerimg)

print(datpower.shape)

(91, 109, 91)


In [4]:
# Sort power coordinates

pcoords = []
proinums = []

for roi in range(264):
    x,y,z = np.where(datpower == roi+1)
    pcoords.append(np.stack([x,y,z,np.ones(x.shape[0])]).T)
    proinums.append(roi*np.ones(x.shape[0]))
    
pcoords = np.concatenate(pcoords).T
proinums = np.concatenate(proinums)

print(pcoords.shape)
print(proinums.shape)

(4, 21384)
(21384,)


In [6]:
from scipy.interpolate import interpn
from scipy import signal

def butter_bandpass(cutoff, fs, order=5):
    normal_cutoff = [cutoff[0], cutoff[1]]
    b, a = signal.butter(order, normal_cutoff, btype='band', analog=False, fs=fs)
    return b, a

def butter_bandpass_filter(data, cutoff, fs, order=5):
    b, a = butter_bandpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

# Extract timeseries quickly

tsdict = dict()

for i,sub in enumerate(longsubs):
    name = sub['name']
    scans = []

    print(f'{i} Starting sub {name}')
    
    for scan in sub['scans']:
        year = int(scan['year'])
        month = int(scan['month'])
        fname = scan['mniscan']
        tr = scan['tr']
    
        print(f'Doing {year} {month} {tr}', end='')
        
        img = image.load_img(fname)
        datimg = image.get_data(img)
        
        coords = np.linalg.inv(img.affine)@powerimg.affine@pcoords
        points = [np.arange(s) for s in img.shape[:-1]]
        
        nt = datimg.shape[-1]
        ts = np.zeros((264,nt))
        res = np.zeros((nt,coords.shape[-1]))

        try:
            for tp in range(nt):
                res[tp] = interpn(points, datimg[:,:,:,tp], coords[:-1].T, bounds_error=False)
        except:
            print('... fail')
            continue

        for roi in range(264):
            ts[roi] = np.mean(res[:,proinums == roi], axis=1)
            
        scan['ts'] = ts

        try:
            if tr < 6:
                tsfilt = butter_bandpass_filter(ts, [0.01, 0.15], 1/tr)
            else:
                tsfilt = butter_bandpass_filter(ts, [0.01, 0.08], 1/tr)
        except:
            print('... fail')
            continue
            
        scan['tsfilt'] = tsfilt
        
        if np.sum(np.isnan(tsfilt)) > 0:
            print(f'... Bad {i} {sub} {year} {month}')

        print(' ok')
    
print('Complete')

0 Starting sub 002_S_0295
Doing 2011 6 6.02158 ok
Doing 2012 5 3.001 ok
1 Starting sub 002_S_0413
Doing 2019 8 0.607 ok
Doing 2011 6 6.02158 ok
Doing 2012 5 3.001 ok
Doing 2013 5 3.001 ok
Doing 2017 6 3 ok
Doing 2014 5 3.004 ok
2 Starting sub 002_S_0685
Doing 2011 7 6.02158 ok
Doing 2012 7 3.001... fail
3 Starting sub 002_S_0729
Doing 2011 8 6.02158 ok
Doing 2012 8 3.001 ok
Doing 2013 8 3.001 ok
4 Starting sub 002_S_1155
Doing 2019 5 0.607 ok
Doing 2011 12 3.001 ok
Doing 2012 12 6.02158 ok
Doing 2014 1 3.001 ok
Doing 2017 4 3 ok
Doing 2018 5 3 ok
Doing 2015 1 6.02302 ok
5 Starting sub 002_S_1261
Doing 2019 5 0.607 ok
Doing 2011 3 3 ok
Doing 2012 2 3.001 ok
Doing 2013 2 3.001 ok
Doing 2017 3 3 ok
Doing 2018 4 3 ok
Doing 2015 6 3.025 ok
6 Starting sub 002_S_1268
Doing 2011 4 3 ok
Doing 2012 3 3.001 ok
7 Starting sub 002_S_1280
Doing 2019 3 0.607 ok
Doing 2011 5 3 ok
Doing 2012 3 3.001 ok
Doing 2013 2 3.001 ok
Doing 2017 3 3 ok
Doing 2014 3 3.004 ok
Doing 2015 5 3.025 ok
8 Starting sub 00

In [7]:
# Dump subs ts and tsfilt to file

import pickle

pickle.dump(longsubs, open('/home/anton/Documents/Tulane/Research/Work/ADNI/long_sub_scans_ts_tsfilt.pkl', 'wb'))

print('Complete')

Complete
